In [34]:
import numpy as np
import pandas as pd

from lightgbm import LGBMClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier)
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [37]:
#Coleta de Dados
train = pd.read_csv('train.csv')
treino = train.drop('Id', axis=1)
test = pd.read_csv('test.csv')

#Split de variáveis
X = treino.iloc[:,0:54]
y = treino['Cover_Type']
X_test = test.drop('Id', axis=1)

#Mantendo as melhores variáveis
X_new = X.loc[:,['Elevation', 'Horizontal_Distance_To_Roadways',
                 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area4', 'Aspect',
                 'Vertical_Distance_To_Hydrology',
                 'Horizontal_Distance_To_Hydrology', 'Hillshade_9am',
                 'Hillshade_3pm', 'Hillshade_Noon', 'Slope', 'Soil_Type2',
                 'Soil_Type10']]

y_new = y.copy()

X_teste = teste.loc[:,['Elevation', 'Horizontal_Distance_To_Roadways',
                 'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area4', 'Aspect',
                 'Vertical_Distance_To_Hydrology',
                 'Horizontal_Distance_To_Hydrology', 'Hillshade_9am',
                 'Hillshade_3pm', 'Hillshade_Noon', 'Slope', 'Soil_Type2',
                 'Soil_Type10']]

In [23]:
stack_model = stack(X_new,y_new,X_test)

> Iniciaizando Classificadores
> Empilhando Classificadores
Fitting 4 classifiers...
Fitting classifier1: adaboostclassifier (1/4)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  6.4min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting classifier2: gradientboostingclassifier (2/4)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 10.2min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting classifier3: lgbmclassifier (3/4)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting classifier4: randomforestclassifier (4/4)


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   29.4s finished


> Criando submission file


In [38]:

def stack(X,y,X_test):

    print('> Iniciaizando Classificadores')
    n_jobs = -1  
    random_state = 2

    #ab_clf = AdaBoostClassifier(n_estimators=1000,
                                #base_estimator=DecisionTreeClassifier(min_samples_leaf=4,min_samples_split =12, criterion = 'entropy',max_depth = 60,
                                                                                        #min_impurity_decrease = 0.01,random_state=random_state),random_state=random_state)

    #gb_clf = GradientBoostingClassifier(n_estimators=1000,min_samples_leaf=3,tol=0.1,verbose=0,random_state=random_state)
    

    lg_clf = LGBMClassifier(n_estimators=1200,num_leaves=50,learning_rate = 0.01, max_depth = 80, objective = 'multiclassova', 
                            verbosity=0,random_state=random_state,n_jobs=n_jobs)
    

    #class_weight = dict({1:3, 2:180, 3:3,4:1,5:1,6:3,7:1})
    rf_clf = RandomForestClassifier(n_estimators=2000,min_samples_leaf=1,min_samples_split = 2, 
                                    max_features = 'sqrt', max_depth = 110, verbose=0,random_state=random_state, n_jobs=n_jobs)

    xg_clf = xgb.XGBClassifier(subsample= 0.8,reg_lambda = 3.0,reg_alpha = 0.6,n_estimators = 2000, min_child_weight = 5,
                        learning_rate = 0.1,gamma = 0.73,colsample_bytree = 1.0, three_method = 'exact', 
                        predictor= 'cpu_predictor', objective="multi:softprob",verbosity=0,random_state=1,n_jobs=n_jobs)

    
     #('gb', gb_clf),('ab', ab_clf)
    
    ensemble = [('lg', lg_clf),
                ('rf', rf_clf),
                ('xg', xg_clf)]
   
    print('> Empilhando Classificadores')
    
    test_ids = test.index.copy()

    stack = StackingCVClassifier(classifiers=[clf for label, clf in ensemble],
                             meta_classifier=lg_clf,
                             cv=5,
                             use_probas=False,
                             use_features_in_secondary=True,
                             verbose=1,
                             random_state=random_state,
                             n_jobs=n_jobs)
    
    stack= stack.fit(X,y)

    p_sub = stack.predict(X_teste)
    sub = pd.DataFrame(p_sub, index = test['Id'], columns=['Cover_Type'])
    print(sub.shape)

    #Salvando em csv
    sub.to_csv('submission.csv')
    print('Arquivo salvo!')
    return stack.meta_clf_